![Team USA Football](images/Usa_football_body_logo.png)

Data Dictionary here https://nflreadr.nflverse.com/articles/dictionary_player_stats.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
stats_df = pd.read_csv('data/player_stats.csv')
qbr_df = pd.read_csv('data/qbr_season_level.csv') 

stats_df contains player stats by game ill need to get some season totals, career totals, as well as per game stats for passing receiving and rushing. ill use interceptions, special teams tds, and fumbles, sacks to help rank defense. the stats ill use arent missing many entries so after cleaning ill save what i need into a new dataframe. also need to have a cut off for age. if a player is XX y/o they wont be relevant today. ill try to expand on the EPA missing data as that is Expected Points Added which should be extremely beneficial to analysis i should also seperate playoffs and regular season.  

In [3]:
stats_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,1,REG,...,0,0.292378,0,0.0,0.052632,NaN,NaN,0,12.7,13.7
1,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,2,REG,...,1,0.377009,0,0.0,0.117647,NaN,NaN,0,5.1,8.1
2,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,4,REG,...,0,-0.699578,0,NaN,0.023810,NaN,NaN,0,0.2,0.2
3,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,7,REG,...,0,-0.228454,0,0.0,0.050000,NaN,NaN,0,3.5,5.5
4,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,8,REG,...,0,NaN,0,NaN,NaN,NaN,NaN,0,3.9,3.9


In [27]:
# Filter stats_df to include only rows where player_display_name is equal to 'Aaron Rodgers'
aaron_rodgers_df = stats_df[stats_df['player_display_name'] == 'Aaron Rodgers']

# Display the filtered DataFrame
aaron_rodgers_df.tail(20)

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,season_passing_yards,season_rushing_yards,season_receiving_yards,season_passing_tds,season_rushing_tds,season_receiving_tds,season_completion_percentage,completions_per_attempt,yards_per_touch,qbr_total
112809,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2021,18,REG,...,4340,101,-4,37,3,0,11.437398,0.736842,3.500000,74.1
112810,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2021,20,POST,...,4340,101,-4,37,3,0,11.437398,0.666667,0.000000,22.6
118474,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2022,1,REG,...,3695,94,0,26,1,0,10.719671,0.628571,-0.500000,41.3
118475,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2022,2,REG,...,3695,94,0,26,1,0,10.719671,0.730769,1.666667,41.3
118476,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2022,3,REG,...,3695,94,0,26,1,0,10.719671,0.750000,-0.500000,41.3
118477,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2022,4,REG,...,3695,94,0,26,1,0,10.719671,0.583333,0.500000,41.3
118478,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2022,5,REG,...,3695,94,0,26,1,0,10.719671,0.625000,0.000000,41.3
118479,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2022,6,REG,...,3695,94,0,26,1,0,10.719671,0.619048,0.000000,41.3
118480,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2022,7,REG,...,3695,94,0,26,1,0,10.719671,0.638889,0.000000,41.3
118481,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,GB,2022,8,REG,...,3695,94,0,26,1,0,10.719671,0.612903,5.500000,41.3


In [24]:
stats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129739 entries, 0 to 129738
Data columns (total 70 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   player_id                     129739 non-null  object 
 1   player_name                   61504 non-null   object 
 2   player_display_name           129736 non-null  object 
 3   position                      129667 non-null  object 
 4   position_group                129667 non-null  object 
 5   headshot_url                  69803 non-null   object 
 6   recent_team                   129739 non-null  object 
 7   season                        129739 non-null  int64  
 8   week                          129739 non-null  int64  
 9   season_type                   129739 non-null  object 
 10  opponent_team                 129739 non-null  object 
 11  completions                   129739 non-null  int64  
 12  attempts                      129739 non-nul

In [6]:
# Use 1 to show all info in cell below then comment it out and use 2 to go back to default view

#1
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)

#2
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
#pd.reset_option('display.max_colwidth')


In [7]:
stats_df['position'].value_counts()

WR     49363
RB     33350
TE     24147
QB     16086
FB      5219
P        459
CB       148
G        128
DB       123
T         94
FS        93
SS        77
OLB       56
LS        56
K         46
LB        45
ILB       32
DE        28
C         28
DT        20
OT        17
DL        15
HB        13
NT        10
MLB        8
OG         3
OL         2
S          1
Name: position, dtype: int64

In [8]:
# Group stats_df by opponent_team and season, then sum the interceptions
interception_data = stats_df.groupby(['opponent_team', 'season'])['interceptions'].sum().reset_index()
interception_data.rename(columns={'opponent_team': 'defense', 'interceptions': 'interceptions'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the sacks
sack_data = stats_df.groupby(['opponent_team', 'season'])['sacks'].sum().reset_index()
sack_data.rename(columns={'opponent_team': 'defense', 'sacks': 'sacks'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the sack_yards
sack_yards_data = stats_df.groupby(['opponent_team', 'season'])['sack_yards'].sum().reset_index()
sack_yards_data.rename(columns={'opponent_team': 'defense', 'sack_yards': 'sack_yards'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the receiving yards
receiving_yards_data = stats_df.groupby(['opponent_team', 'season'])['receiving_yards'].sum().reset_index()
receiving_yards_data.rename(columns={'opponent_team': 'defense', 'receiving_yards': 'receiving_yards_allowed'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the rushing yards
rushing_yards_data = stats_df.groupby(['opponent_team', 'season'])['rushing_yards'].sum().reset_index()
rushing_yards_data.rename(columns={'opponent_team': 'defense', 'rushing_yards': 'rushing_yards_allowed'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the rushing tds
rushing_tds_data = stats_df.groupby(['opponent_team', 'season'])['rushing_tds'].sum().reset_index()
rushing_tds_data.rename(columns={'opponent_team': 'defense', 'rushing_tds': 'rushing_tds_allowed'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the receiving tds
receiving_tds_data = stats_df.groupby(['opponent_team', 'season'])['receiving_tds'].sum().reset_index()
receiving_tds_data.rename(columns={'opponent_team': 'defense', 'receiving_tds': 'receiving_tds_allowed'}, inplace=True)

# Group stats_df by recent_team and season, then sum the special teams tds
special_teams_tds_data = stats_df.groupby(['recent_team', 'season'])['special_teams_tds'].sum().reset_index()
special_teams_tds_data.rename(columns={'recent_team': 'defense', 'special_teams_tds': 'defense_special_teams_tds'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the sack fumbles
sack_fumbles_data = stats_df.groupby(['opponent_team', 'season'])['sack_fumbles'].sum().reset_index()
sack_fumbles_data.rename(columns={'opponent_team': 'defense', 'sack_fumbles': 'sack_fumbles_forced'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the rushing fumbles
rushing_fumbles_data = stats_df.groupby(['opponent_team', 'season'])['rushing_fumbles'].sum().reset_index()
rushing_fumbles_data.rename(columns={'opponent_team': 'defense', 'rushing_fumbles': 'rushing_fumbles_forced'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the receiving fumbles
receiving_fumbles_data = stats_df.groupby(['opponent_team', 'season'])['receiving_fumbles'].sum().reset_index()
receiving_fumbles_data.rename(columns={'opponent_team': 'defense', 'receiving_fumbles': 'receiving_fumbles_forced'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the sack fumbles recovered
sack_fumbles_lost_data = stats_df.groupby(['opponent_team', 'season'])['sack_fumbles_lost'].sum().reset_index()
sack_fumbles_lost_data.rename(columns={'opponent_team': 'defense', 'sack_fumbles_lost': 'sack_fumbles_recovered'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the rushing fumbles recovered
rushing_fumbles_lost_data = stats_df.groupby(['opponent_team', 'season'])['rushing_fumbles_lost'].sum().reset_index()
rushing_fumbles_lost_data.rename(columns={'opponent_team': 'defense', 'rushing_fumbles_lost': 'rushing_fumbles_recovered'}, inplace=True)

# Group stats_df by opponent_team and season, then sum the receiving fumbles recovered
receiving_fumbles_lost_data = stats_df.groupby(['opponent_team', 'season'])['receiving_fumbles_lost'].sum().reset_index()
receiving_fumbles_lost_data.rename(columns={'opponent_team': 'defense', 'receiving_fumbles_lost': 'receiving_fumbles_recovered'}, inplace=True)

# Merge the new data into a single defense_df
defense_df = pd.merge(interception_data, sack_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, sack_yards_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, rushing_yards_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, receiving_yards_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, rushing_tds_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, receiving_tds_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, special_teams_tds_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, sack_fumbles_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, rushing_fumbles_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, receiving_fumbles_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, sack_fumbles_lost_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, rushing_fumbles_lost_data, on=['defense', 'season'], how='outer')
defense_df = pd.merge(defense_df, receiving_fumbles_lost_data, on=['defense', 'season'], how='outer')

# Check the updated DataFrame
defense_df.head()

,defense,season,interceptions,sacks,sack_yards,rushing_yards_allowed,receiving_yards_allowed,rushing_tds_allowed,receiving_tds_allowed,defense_special_teams_tds,sack_fumbles_forced,rushing_fumbles_forced,receiving_fumbles_forced,sack_fumbles_recovered,rushing_fumbles_recovered,receiving_fumbles_recovered
0,ARI,1999,17,33,229,2241,3372,17,25,0,8,9,6,3,3,2
1,ARI,2000,8,24,126,2549,3237,29,19,1,2,7,5,1,5,4
2,ARI,2001,17,19,128,2087,3726,10,26,0,1,12,2,1,2,0
3,ARI,2002,17,21,131,2146,4005,17,29,1,2,5,5,1,2,3
4,ARI,2003,13,21,97,1915,3686,17,29,1,5,11,3,2,6,2


In [9]:
#drop old seasons but not yet find a good place for this cell
#stats_df = stats_df[stats_df['season'].isin(range(2014, 2024))]

In [10]:
#seperate reg and post season data not sure where to put this either
#reg_season_df = stats_df[stats_df['season_type'] == 'REG']
#post_season_df = stats_df[stats_df['season_type'] == 'POST']

### Feature Engineering

ill save the following cell as a function to use for post_season_df idk maybe

In [11]:
# Create efficiency metrics
stats_df['completion_percentage'] = stats_df['completions'] / (stats_df['attempts'] + 1)
stats_df['yards_per_attempt'] = stats_df['passing_yards'] / (stats_df['attempts'] + 1)
stats_df['yards_per_carry'] = stats_df['rushing_yards'] / (stats_df['carries'] + 1)
stats_df['yards_per_reception'] = stats_df['receiving_yards'] / (stats_df['receptions'] + 1)

# Create touchdown ratios
stats_df['pass_tds_per_attempt'] = stats_df['passing_tds'] / (stats_df['attempts'] + 1)
stats_df['tds_per_carry'] = stats_df['rushing_tds'] / (stats_df['carries'] + 1)
stats_df['tds_per_reception'] = stats_df['receiving_tds'] / (stats_df['receptions'] + 1)

# Calculate season stats for each player over the season
stats_df['season_passing_yards'] = stats_df.groupby(['player_id', 'season'])['passing_yards'].transform('sum')
stats_df['season_rushing_yards'] = stats_df.groupby(['player_id', 'season'])['rushing_yards'].transform('sum')
stats_df['season_receiving_yards'] = stats_df.groupby(['player_id', 'season'])['receiving_yards'].transform('sum')
stats_df['season_passing_tds'] = stats_df.groupby(['player_id', 'season'])['passing_tds'].transform('sum')
stats_df['season_rushing_tds'] = stats_df.groupby(['player_id', 'season'])['rushing_tds'].transform('sum')
stats_df['season_receiving_tds'] = stats_df.groupby(['player_id', 'season'])['receiving_tds'].transform('sum')
stats_df['season_completion_percentage'] = stats_df.groupby(['player_id', 'season'])['completion_percentage'].transform('sum')

# Interaction terms
stats_df['completions_per_attempt'] = stats_df['completions'] / (stats_df['attempts'] + 1)
stats_df['yards_per_touch'] = (stats_df['rushing_yards'] + stats_df['receiving_yards']) / (stats_df['carries'] + stats_df['receptions'] + 1) 

# CAREER STATS

# USE OPPOSING TEAM TO RECORD D/ST STATS

# SEPERATE BY POSITION OR GROUP

In [25]:
# checking out well known *cough JETS cough* players with new features
print(stats_df[stats_df['player_display_name'] == 'Aaron Rodgers'])

#stats_df[stats_df['player_display_name'] == 'Garrett Wilson'].head()

#stats_df[stats_df['player_display_name'] == 'Breece Hall'].head()


         player_id player_name player_display_name position position_group  \
34491   00-0023459   A.Rodgers       Aaron Rodgers       QB             QB   
34492   00-0023459   A.Rodgers       Aaron Rodgers       QB             QB   
34493   00-0023459   A.Rodgers       Aaron Rodgers       QB             QB   
38709   00-0023459   A.Rodgers       Aaron Rodgers       QB             QB   
38710   00-0023459   A.Rodgers       Aaron Rodgers       QB             QB   
...            ...         ...                 ...      ...            ...   
118487  00-0023459   A.Rodgers       Aaron Rodgers       QB             QB   
118488  00-0023459   A.Rodgers       Aaron Rodgers       QB             QB   
118489  00-0023459   A.Rodgers       Aaron Rodgers       QB             QB   
118490  00-0023459   A.Rodgers       Aaron Rodgers       QB             QB   
124086  00-0023459   A.Rodgers       Aaron Rodgers       QB             QB   

                                             headshot_url recen

## QBR

not sure if this will even be necessary. might be good to have QBR by season to go along with other QB stats

In [13]:
qbr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1363 entries, 0 to 1362
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   season         1363 non-null   int64  
 1   season_type    1363 non-null   object 
 2   game_week      1363 non-null   object 
 3   team_abb       1363 non-null   object 
 4   player_id      1363 non-null   int64  
 5   name_short     1363 non-null   object 
 6   rank           775 non-null    float64
 7   qbr_total      1363 non-null   float64
 8   pts_added      1363 non-null   float64
 9   qb_plays       1363 non-null   int64  
 10  epa_total      1363 non-null   float64
 11  pass           1363 non-null   float64
 12  run            1363 non-null   float64
 13  exp_sack       1363 non-null   int64  
 14  penalty        1363 non-null   float64
 15  qbr_raw        1363 non-null   float64
 16  sack           1363 non-null   float64
 17  name_first     1363 non-null   object 
 18  name_las

In [17]:
# Define the columns to keep
columns_to_keep = ['season', 'season_type', 'qbr_total', 'name_display']

# Drop columns from qbr_df except the specified ones
qbr_df = qbr_df[columns_to_keep]

# Replace 'Regular' with 'REG' and 'Playoffs' with 'POST' in the 'season_type' column
qbr_df['season_type'] = qbr_df['season_type'].replace({'Regular': 'REG', 'Playoffs': 'POST'})

# Display the modified qbr_df
qbr_df.head()

,season,season_type,qbr_total,name_display
0,2006,REG,86.4,Peyton Manning
1,2006,REG,68.6,Tom Brady
2,2006,REG,67.4,Philip Rivers
3,2006,REG,67.1,Carson Palmer
4,2006,REG,66.7,Drew Brees


REG     1062
POST     301
Name: season_type, dtype: int64

In [22]:
# Assuming stats_df and qbr_df are your DataFrames
# Define the mapping between columns in stats_df and qbr_df
column_mapping = {'player_display_name': 'name_display', 'season': 'season', 'season_type': 'season_type'}

# Merge DataFrames using the mapped columns
stats_df = pd.merge(stats_df, qbr_df, left_on=['player_display_name', 'season', 'season_type'], 
                     right_on=['name_display', 'season', 'season_type'], how='left')

# Drop the redundant 'name_display' column
stats_df.drop(columns=['name_display'], inplace=True)

# Display the merged DataFrame
stats_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,season_passing_yards,season_rushing_yards,season_receiving_yards,season_passing_tds,season_rushing_tds,season_receiving_tds,season_completion_percentage,completions_per_attempt,yards_per_touch,qbr_total
0,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,1,REG,...,0,443,84,0,1,1,0.0,0.0,3.722222,NaN
1,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,2,REG,...,0,443,84,0,1,1,0.0,0.0,3.923077,NaN
2,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,4,REG,...,0,443,84,0,1,1,0.0,0.0,0.500000,NaN
3,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,7,REG,...,0,443,84,0,1,1,0.0,0.0,3.888889,NaN
4,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,8,REG,...,0,443,84,0,1,1,0.0,0.0,2.785714,NaN
